In [1]:
from google.colab import drive
drive.mount('/content/drive')

# put folder name here
FOLDERNAME = 'multiclass_polyact/'

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

import os
from utils import *
%load_ext autoreload
%autoreload 2

Mounted at /content/drive
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
!pip install ucimlrepo

In [3]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
breast_cancer_wisconsin_original = fetch_ucirepo(id=15)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_original.data.features
y = breast_cancer_wisconsin_original.data.targets
X = X.fillna(0)

train_frac = 0.8
x_np = X.to_numpy()

# convert y vector of 2's and 4's to a vector of 1's and -1's.
y_np = 2 * ((y.to_numpy().squeeze() - 2) / 2) - 1

n,d = x_np.shape
xtrain, ytrain = x_np[:int(train_frac*n), :], y_np[:int(train_frac*n)]
xtest, ytest = x_np[int(train_frac*n):, :], y_np[int(train_frac*n):]
mean, std = xtrain.mean(axis=0, keepdims = True), xtrain.std(axis=0, keepdims=True)
xtrain = (xtrain - mean)/(std + 1e-9)
xtest = (xtest - mean)/(std + 1e-9)

def form_robust_psd_matrix_numpy(Z_tilde, lbda_i, w_i, x_i, y_i, r, d_r,a=0.09,b=0.5,c=0.47):
  Z_1, Z_2, Z_4 = Z_tilde[:-1,:-1], Z_tilde[:-1,-1:], Z_tilde[d_r-1:d_r,d_r-1:d_r]
  no_lambda_block = np.zeros((d_r+1,d_r+1))

  b1 = cp.hstack((a*Z_1, a*Z_1 @ x_i.T - 0.5*b*Z_2))
  b2 =  cp.hstack(((a*Z_1 @ x_i.T - 0.5*b*Z_2).T, cp.reshape(forward_cp(x_i, Z_tilde) - w_i*y_i, (1,1))))
  no_lambda_block = cp.vstack((b1,b2))

  lambda_block = np.eye(no_lambda_block.shape[0])
  lambda_block[-1,d_r:d_r+1] = -r.value**2
  return lbda_i*lambda_block + y_i * no_lambda_block

def forward_cp(X, Z_tilde_sym, a=0.09, b=0.5, c=0.47):
  return a * cp.sum(cp.multiply(X, X @ Z_tilde_sym[:-1,:-1]), axis=1) + b * X @ Z_tilde_sym[:-1,-1] + c * Z_tilde_sym[-1,-1]

def cvx_prob(x,y,r,beta, verbose=False, tol=1e-5,idxs=None, warm_start=False):
  if idxs is None:
    idxs = np.arange(x.shape[0])
  n, d = x.shape
  Z = cp.Variable((d + 1, d + 1), symmetric=True)
  Zp = cp.Variable((d + 1, d + 1), symmetric=True)
  w = cp.Variable(n)
  lbda = cp.Variable(n)

  constraints = []
  for i in idxs:
    constraints =constraints + [form_robust_psd_matrix_numpy(Z-Zp, lbda[i], w[i], x[i:i+1], y[i], r, d) >> 0]
  constraints =constraints + [lbda >= 0, cp.trace(Z[:-1,:-1]) == Z[-1,-1], cp.trace(Zp[:-1,:-1]) == Zp[-1,-1], Z >>0, Zp >> 0]
  prob = cp.Problem(cp.Minimize(cp.sum(cp.pos(1 - w)) + beta*(Z[-1,-1] + Zp[-1,-1])), constraints)
  prob.solve(solver=cp.SCS, verbose=verbose, eps=tol, warm_start=warm_start)
  return prob, Z, Zp, w, lbda

def cvx_nonrobust(x,y,beta):
  n, d = x.shape
  Z = cp.Variable((d + 1, d + 1), symmetric=True)
  Zp = cp.Variable((d + 1, d + 1), symmetric=True)

  constraints = [cp.trace(Z[:-1,:-1]) == Z[-1,-1], cp.trace(Zp[:-1,:-1]) == Zp[-1,-1], Z >> 0, Zp >> 0]
  out = forward_cp(x, Z-Zp)
  prob = cp.Problem(cp.Minimize(cp.mean(cp.pos(1 - cp.multiply(y,out))) + beta*(Z[-1,-1] + Zp[-1,-1])), constraints)
  prob.solve()
  return prob, Z, Zp

def evaluate_robust_accuracies(X, y, Z_tilde, device, magnitudes=0.1*np.arange(7)):
  out = {}
  for attack_size in magnitudes:
    out[attack_size] = accuracy(y, forward(X + fgsm(y, X, Z_tilde, device, magnitude=attack_size, order=np.inf), Z_tilde))
    print(round(attack_size, 3), ':', round(out[attack_size], 4))
  return out

def flip_distance(Z_tilde, x, label):
    a,b,c = 0.09, 0.5, 0.47
    d = x.shape[1]
    gamma, s = cp.Variable((1,1)), cp.Variable((1, 1))
    A0, b0, c0 = -np.eye(x.shape[1]), x.T, -np.linalg.norm(x)**2
  
    A1 = label * a*(Z_tilde[:-1,:-1])
    b1 = label * (b/2)*(Z_tilde[:-1,-1])[:,None]
    c1 = label * c*Z_tilde[-1,-1][None][None]
    print(A1.shape, c1.shape, b1.shape)
    block1 = cp.bmat([[A0, b0], [b0.T, c0 + s]])
    block2 = gamma * cp.bmat([[A1, b1],[b1.T, c1]])
    constraints = [block2 - block1 >> 0, gamma >= 0]
    obj = cp.Maximize(s)
    prob = cp.Problem(obj,constraints)
    dist = prob.solve(solver=cp.SCS)
    return np.sqrt(dist)

In [4]:
tot = 0
count = 0
out = []
for x1 in xtrain[ytrain == 1]:
  for x2 in xtrain[ytrain == -1]:
    cur = np.linalg.norm(x1-x2)
    out.append(cur)
    tot += cur
    count += 1

print(tot/count)

5.309603108171895


In [ ]:
beta = 0.01
robust_sizes = np.linspace(0.4,0.5,2)
avg_dists = []
clean_accs = []
robust_accs = []
clean = 0.0
robust = 0.8
warm_start = False
for robust_radius in robust_sizes[:20]: #robust_sizes:
  print(robust_radius)
  r = cp.Parameter(1)
  r.value = np.array([robust_radius])
  prob, Z, Zp, w, lbda = cvx_prob(xtrain, ytrain, r, beta, tol=1e-5, verbose=False, warm_start=warm_start)
  warm_start = True

  Z_tilde = torch.tensor((Z-Zp).value)

  out_dict = evaluate_robust_accuracies(torch.tensor(xtest), torch.tensor(ytest), Z_tilde, 'cpu', magnitudes = np.array([clean, robust]))

  clean_accs.append(out_dict[clean])
  robust_accs.append(out_dict[robust])

  count_robust = 0
  count_nonrobust = 0
  dists_robust = []


  for i, y in enumerate(ytest):
    curx = xtest[i:i+1]
    out_r = forward(torch.tensor(curx), Z_tilde)
    if np.sign(out_r) == y:
      count_robust += 1
      dist = flip_distance(Z_tilde.numpy(), curx, y)
      dists_robust.append(dist)

  avg_dists.append(sum(dists_robust)/count_robust)

In [ ]:
nonrobust_p, Z_nonrobust, Zp_nonrobust = cvx_nonrobust(xtrain, ytrain, beta)
Z_tilde_nonrobust =torch.tensor((Z_nonrobust- Zp_nonrobust).value)
dists_nonrobust = []
for i, y in enumerate(ytest):
    curx = xtest[i:i+1]
    out_r = forward(torch.tensor(curx), Z_tilde_nonrobust)
    if np.sign(out_r) == y:
      count_robust += 1
      dist = flip_distance(Z_tilde_nonrobust.numpy(), curx, y)
      dists_nonrobust.append(dist)
avg_dists_nonrobust = sum(dists_nonrobust)/len(dists_nonrobust)

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(12,3))
font_dict = {'fontsize': 14}
axs[0].plot(robust_sizes[:16], avg_dists[:16],'-', color='red', linewidth=2,label='Adversarial')
axs[0].hlines(avg_dists_nonrobust,robust_sizes[0], robust_sizes[16], linestyles='--', linewidth=2, color='black', label=r'Standard' )
axs[0].set_xlabel(r'Robust radius parameter $r$', **font_dict)
axs[0].set_ylabel(r'$d_\mathcal{D}(x)$', **font_dict)
axs[0].set_xlim([robust_sizes[0], robust_sizes[15]])

axs[1].plot(robust_sizes[:20], clean_accs[:20], '-', color='blue', linewidth=2, label='Clean Adversarial')
axs[1].plot(robust_sizes[:20], robust_accs[:20], '--', color='blue', linewidth=2,label=r'Robust Adversarial')
axs[1].hlines(nonrobust_fgsm_acc[robust], robust_sizes[0], robust_sizes[20], linestyles='--',linewidth=2,color='black', label = r'Robust Standard')
axs[1].set_xlabel(r'Robust radius parameter $r$', **font_dict)
axs[1].set_xlim([robust_sizes[0], robust_sizes[19]])
axs[1].set_ylabel("Test accuracy", **font_dict)

axs[0].legend(**font_dict, loc='upper left')
axs[1].legend(**font_dict, loc = 'center right', bbox_to_anchor=(1, 0.37), ncol=1)#, loc='center right')
fig.autofmt_xdate()
plt.savefig('BCW_dist_plot.png')